## 0. Libarary 불러오기 및 경로설정

In [78]:
import os
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

import torch.optim as optim
import tqdm

In [79]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

device:[cuda:0].


In [80]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'
train_dir = '/opt/ml/input/data/train'

In [81]:
# meta 데이터와 이미지 경로를 불러옵니다.
# 7 : 3 비율로 나눕니다.
submission = pd.read_csv(os.path.join(train_dir, 'train_label.csv'))
#submission_train = submission[:]#int(len(submission)*0.7)]
#submission_test = submission[:]#int(len(submission)*0.7):]
submission_train = submission.sample(frac=0.7)
submission_test = submission.drop(submission_train.index)

print(len(submission),len(submission_test)+len(submission_train))

18900 18900


In [82]:
class TrainDataset(Dataset):
    def __init__(self, train_idx, transform):
        self.train_idx = train_idx
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.train_idx[index][0])

        if self.transform:
            image = self.transform(image)
        return image, train_idx[index][1]

    def __len__(self):
        return len(self.train_idx)

class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [83]:

image_dir = os.path.join(train_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
# image_paths = [os.path.join(image_dir, img_id) for img_id in submission.path]
train_idx = [i for i in zip(submission_train.path,submission_train.label)]
transform = transforms.Compose([
    transforms.CenterCrop(256),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
train_dataset = TrainDataset(train_idx, transform)
train_loader = DataLoader(
    train_dataset,
    batch_size = 64,
    shuffle=True,
    num_workers=4
)

test_idx = [i for i in zip(submission_test.path,submission_test.label)]
transform = transforms.Compose([
    transforms.CenterCrop(256),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
test_dataset = TrainDataset(test_idx, transform)
test_loader = DataLoader(
    test_dataset,
    batch_size = 64,
    shuffle=True,
    num_workers=4
)

## 1. Model 정의

In [ ]:
!pip install ipywidgets
!conda install -c conda-forge ipywidgets

In [95]:
imagenet_resnet18 = torchvision.models.resnet18(pretrained=True)

In [96]:
MNIST_CLASS_NUM = 18 # MNIST CLASS NUM은 몇 개일까요?
imagenet_resnet18.fc = torch.nn.Linear(in_features=512, out_features=MNIST_CLASS_NUM, bias=True) # resnet18.fc의 in_features의 크기는?
torch.nn.init.xavier_uniform_(imagenet_resnet18.fc.weight)
stdv = 1.0/np.sqrt(512) # fully connected layer의 bias를 resnet18.fc in_feature의 크기의 1/root(n) 크기의 uniform 분산 값 중 하나로 설정해주세요! - Why? https://stackoverflow.com/questions/49433936/how-to-initialize-weights-in-pytorch
imagenet_resnet18.fc.bias.data.uniform_(-stdv, stdv)

tensor([ 0.0106, -0.0173,  0.0372, -0.0116,  0.0073,  0.0031,  0.0058,  0.0402,
        -0.0320,  0.0304,  0.0429, -0.0091,  0.0103,  0.0280, -0.0056, -0.0425,
        -0.0204,  0.0050])

In [97]:
# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
target_model = imagenet_resnet18
target_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
#입력 크기 테스트
from torchsummary import summary
summary(imagenet_resnet18, input_size=(3, 256, 256), device=device.type) 

---

In [86]:
def func_eval(model,data_iter,device):
    with torch.no_grad():
        n_total,n_correct = 0,0
        model.eval() # evaluate (affects DropOut and BN)
        for image, target in data_iter:
            y_trgt = target.to(device)
            model_pred = model(image.to(device))
            _,y_pred = torch.max(model_pred.data,1)
            n_correct += (y_pred==y_trgt).sum().item()
            n_total += image.size(0)
        val_accr = (n_correct/n_total)
        model.train() # back to train mode 
    return val_accr
print ("Done")

Done


In [30]:
train_accr = func_eval(target_model,train_loader,device)

---

In [98]:
## 2. mnist train 데이터 셋을 resnet18 모델에 학습하기

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 5 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

dataloaders = {
    "train" : train_loader,
    "test" : test_loader
}

cuda:0 is using!


In [93]:
from tqdm.notebook import tqdm # tqdm이라는 "반복문"의 현재 진행 상태를 progress-bar로 보여주는 라이브러리

In [99]:
### 학습 코드 시작
best_test_accuracy = 0.
best_test_loss = 9999.
for epoch in range(NUM_EPOCH):
  for phase in ["train", "test"]:
    running_loss = 0.
    running_acc = 0.
    if phase == "train":
      target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "test":
      target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
      # hint - with, pbar
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = target_model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc
    if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
      best_test_loss = epoch_loss
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")


현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.484, 평균 Accuracy : 0.852



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 8.162, 평균 Accuracy : 0.131



현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.086, 평균 Accuracy : 0.977



현재 epoch-1의 test-데이터 셋에서 평균 Loss : 9.504, 평균 Accuracy : 0.129



현재 epoch-2의 train-데이터 셋에서 평균 Loss : 0.022, 평균 Accuracy : 0.996



현재 epoch-2의 test-데이터 셋에서 평균 Loss : 10.689, 평균 Accuracy : 0.129



현재 epoch-3의 train-데이터 셋에서 평균 Loss : 0.009, 평균 Accuracy : 0.999



현재 epoch-3의 test-데이터 셋에서 평균 Loss : 11.001, 평균 Accuracy : 0.128



현재 epoch-4의 train-데이터 셋에서 평균 Loss : 0.005, 평균 Accuracy : 1.000



현재 epoch-4의 test-데이터 셋에서 평균 Loss : 11.718, 평균 Accuracy : 0.129
학습 종료!
최고 accuracy : 0.13068783283233643, 최고 낮은 loss : 8.16239436787062


## 3. Inference

In [46]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    transforms.CenterCrop(256),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

test_dataset = TestDataset(image_paths, transform)

test_loader = DataLoader(
    test_dataset,
    batch_size = 64,
    shuffle=True,
    num_workers=2
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
target_model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in test_loader:
    with torch.no_grad():
        images = images.to(device)
        pred = target_model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!
